1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 백터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에서 유사도 검사
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [2]:
%pip install --upgrade --quiet docx2txt langchain-community

  DEPRECATION: docx2txt is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # LLM은 토큰 수를 계산, 청크사이즈는 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    # 오버랩은 겹치는 토큰 수, 오버랩을 주면서 유사도를 높일 수 있음
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [24]:
len(document_list)

218

In [45]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [46]:
from langchain_chroma import Chroma

# 임베딩한 내용을 벡터 데이터베이스에 저장
# 인메모리 DB이기에 창을 닫으면 날라간다. 옵션에 persist_directory를 주면 파일로 저장할 수 있다.
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory = './chroma_db',
#     collection_name = 'chroma_tax'
# )
database = Chroma(embedding_function=embedding, collection_name = 'chroma_tax', persist_directory = './chroma_db' )

In [47]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3)

In [48]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o')

In [29]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question : {query}
# """

In [30]:
# ai_message = llm.invoke(prompt)

In [20]:
# ai_message

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 다음 단계를 거쳐야 합니다.\n\n1. **총급여 계산**:\n   - 연봉 5천만원은 총급여액입니다.\n\n2. **근로소득공제**:\n   - 근로소득공제는 총급여에 따라 계산됩니다.\n   - 총급여액이 5천만원일 경우, 근로소득공제는 다음과 같이 계산됩니다:\n     - 5천만원 이하의 경우: 550만원 + (총급여액 - 5천만원) × 15%\n     - 5천만원을 초과하지 않으므로 550만원이 그대로 적용됩니다.\n\n3. **과세표준 계산**:\n   - 과세표준 = 총급여액 - 근로소득공제 - 기본공제 (1인당 150만원) - 기타 인적공제 및 추가 공제\n   - 기본공제는 본인 150만원이므로, 추가 공제는 없다고 가정하면 과세표준은:\n     - 5천만원 - 550만원 - 150만원 = 4,300만원\n\n4. **소득세율 적용**:\n   - 과세표준 4,300만원에 대해 기본세율을 적용합니다.\n   - 소득세율은 다음과 같습니다:\n     - 1,200만원 이하: 6%\n     - 1,200만원 초과 4,600만원 이하: 15%\n     - 4,600만원 초과 8,800만원 이하: 24%\n     - 8,800만원 초과 1억5천만원 이하: 35%\n     - 1억5천만원 초과 3억원 이하: 38%\n     - 3억원 초과 5억원 이하: 40%\n     - 5억원 초과: 42%\n\n   - 과세표준 4,300만원에 대한 소득세 계산은 다음과 같습니다:\n     - 1,200만원 × 6% = 72만원\n     - (4,300만원 - 1,200만원) × 15% = 3,100만원 × 15% = 465만원\n     - 따라서 총 소득세는 72만원 + 465만원 = 537만원\n\n5. **세액공제**:\n   - 근로소득 세액공제 등을 적용하여 최종 소득세를 계산합니다.\n   - 근로소득 세액공제는 다음과 같습니다:\n     - 총급

In [54]:
from langchain import hub

import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('LANGCHAIN_API_KEY')
prompt = hub.pull('rlm/rag-prompt', api_key=api_key)

/Users/baegmyeongsang/.pyenv/versions/3.10.14/envs/llm-app/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [57]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt" : prompt}
)

In [58]:
ai_message = qa_chain({"query" : query})

/Users/baegmyeongsang/.pyenv/versions/3.10.14/envs/llm-app/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  _alternative_import: str = alternative_import,


In [59]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율이 적용됩니다. 기본세율은 소득 구간에 따라 다르므로, 구체적인 소득세 금액을 계산하려면 추가적인 정보가 필요합니다. 정확한 소득세 금액을 계산하려면 국세청의 소득세 계산기를 이용하거나 세무사와 상담하는 것이 좋습니다.'}